<a href="https://colab.research.google.com/github/donghwi2022/ds-sa-cp2-phase2/blob/%EC%8B%A0%EB%8F%99%ED%9C%98/load_annoy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 import 및 사용 데이터 load

In [1]:
!pip install --user annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import annoy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data = pd.read_parquet("/content/drive/MyDrive/cp2_phase2_data/fashion_logs.parquet")
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1201009 entries, 0 to 1254584
Data columns (total 23 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   created_at          1201009 non-null  category
 1   session_id          1201009 non-null  int32   
 2   payment_method      1201009 non-null  category
 3   promo_amount        1201009 non-null  int32   
 4   promo_code          1201009 non-null  category
 5   shipment_fee        1201009 non-null  int32   
 6   total_amount        1201009 non-null  int32   
 7   product_id          1201009 non-null  int32   
 8   productDisplayName  1201009 non-null  category
 9   quantity            1201009 non-null  int32   
 10  item_price          1201009 non-null  int32   
 11  product_gender      1201009 non-null  category
 12  baseColor           1201009 non-null  category
 13  season              1201009 non-null  category
 14  year                1201009 non-null  int32   
 15

In [5]:
data.head()

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
0,2018-07-29T15:22:01,194913,Debit Card,1415,WEEKENDSERU,10000,199832,54728,Vans Men Black Shoes,1,...,Summer,2012,Casual,Footwear Shoes Casual Shoes,5868,F,2000-08-20,Android,Jakarta Raya,2018-07-28
1,2018-07-30T12:40:22,155874,Credit Card,0,No,10000,155526,16193,Puma Men Knitted Vest Green Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
2,2018-09-15T11:51:17,489413,OVO,0,No,10000,550696,53686,Kiara Women Purple & Yellow Handbag,4,...,Summer,2012,Casual,Accessories Bags Handbags,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
3,2018-11-01T11:23:48,627839,Credit Card,0,No,0,271012,20228,Wrangler Women Cable Red Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
4,2018-12-18T11:20:30,742846,Credit Card,0,No,0,198753,55220,Lakme Absolute Forever Silk Chestnut Lip Liner 03,1,...,Spring,2017,Casual,Personal Care Lips Lip Liner,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28


# 2. CB(annoy) 모델 로드 및 확인

## 1. 사용 데이터 선별



In [6]:
# 사용할 feature 선별
product_data = data[['product_id', 'product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']].drop_duplicates('product_id')
product_data[['product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']] = product_data[['product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']].astype('str')
product_data.reset_index(drop=True, inplace=True)
product_data

,product_id,product_gender,baseColor,season,year,usage,Category
0,54728,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes
1,16193,Men,Green,Fall,2011,Casual,Apparel Topwear Sweaters
2,53686,Women,Purple,Summer,2012,Casual,Accessories Bags Handbags
3,20228,Women,Red,Fall,2011,Casual,Apparel Topwear Sweaters
4,55220,Women,Brown,Spring,2017,Casual,Personal Care Lips Lip Liner
...,...,...,...,...,...,...,...
44440,42430,Women,Blue,Summer,2012,Ethnic,Apparel Topwear Kurtis
44441,59916,Women,Brown,Summer,2012,Ethnic,Apparel Topwear Kurtas
44442,2503,Men,Brown,Fall,2012,Casual,Footwear Shoes Casual Shoes
44443,36651,Women,Red,Summer,2012,Casual,Accessories Wallets Wallets


In [7]:
product_data['features'] = product_data['product_gender']+' ' +product_data['baseColor']+' ' +product_data['season']+' ' +product_data['year']+' ' +product_data['usage']+' ' +product_data['Category']
product_data

,product_id,product_gender,baseColor,season,year,usage,Category,features
0,54728,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
1,16193,Men,Green,Fall,2011,Casual,Apparel Topwear Sweaters,Men Green Fall 2011 Casual Apparel Topwear Swe...
2,53686,Women,Purple,Summer,2012,Casual,Accessories Bags Handbags,Women Purple Summer 2012 Casual Accessories Ba...
3,20228,Women,Red,Fall,2011,Casual,Apparel Topwear Sweaters,Women Red Fall 2011 Casual Apparel Topwear Swe...
4,55220,Women,Brown,Spring,2017,Casual,Personal Care Lips Lip Liner,Women Brown Spring 2017 Casual Personal Care L...
...,...,...,...,...,...,...,...,...
44440,42430,Women,Blue,Summer,2012,Ethnic,Apparel Topwear Kurtis,Women Blue Summer 2012 Ethnic Apparel Topwear ...
44441,59916,Women,Brown,Summer,2012,Ethnic,Apparel Topwear Kurtas,Women Brown Summer 2012 Ethnic Apparel Topwear...
44442,2503,Men,Brown,Fall,2012,Casual,Footwear Shoes Casual Shoes,Men Brown Fall 2012 Casual Footwear Shoes Casu...
44443,36651,Women,Red,Summer,2012,Casual,Accessories Wallets Wallets,Women Red Summer 2012 Casual Accessories Walle...


In [8]:
# tf-idf 벡터화
tfidf = TfidfVectorizer()
product_tfidf = tfidf.fit_transform(product_data['features'])
product_tfidf = pd.DataFrame(product_tfidf.todense(), columns = tfidf.get_feature_names_out())
product_tfidf

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,...,waistcoat,wallets,wash,watches,water,white,winter,women,wristbands,yellow
0,0.0,0.0,0.0,0.0,0.000000,0.257638,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.251266,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.243786,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.227443,0.0,0.0
3,0.0,0.0,0.0,0.0,0.252014,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.216338,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.130091,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44440,0.0,0.0,0.0,0.0,0.000000,0.226641,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.211448,0.0,0.0
44441,0.0,0.0,0.0,0.0,0.000000,0.256879,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.239658,0.0,0.0
44442,0.0,0.0,0.0,0.0,0.000000,0.240922,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
44443,0.0,0.0,0.0,0.0,0.000000,0.177587,0.0,0.0,0.0,0.0,...,0.0,0.861378,0.0,0.0,0.0,0.0,0.0,0.165682,0.0,0.0


## 2. annoy 모델 load

In [9]:
load_annoy = annoy.AnnoyIndex(f=product_tfidf.shape[1], metric='angular')
load_annoy.load('/content/drive/MyDrive/cp2_phase2_data/product.ann')

True

## 3. 샘플 데이터로 확인

In [11]:
# 인덱스가 0인 상품(product_id=54728)에 대해 유사한 상품 20개 추천
product_vec = product_tfidf.iloc[0, :]
get_nns_list = load_annoy.get_nns_by_vector(vector=product_vec, n=21, include_distances=False) # 유사한 벡터를 찾기 때문에 본인 포함
product_data.iloc[get_nns_list[1:]]

,product_id,product_gender,baseColor,season,year,usage,Category,features
560,38548,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
2269,49060,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
2575,26618,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
2706,39640,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
5178,43881,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
5240,42802,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
5474,29568,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
5567,38525,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
6078,40468,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
7288,35660,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
